<a href="https://colab.research.google.com/github/Munima/NLP-tasks/blob/master/corenlp_all_i_need.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mounting googlr drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Install stanfordnlp; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanfordnlp

# Import stanfordnlp
import stanfordnlp

     |████████████████████████████████| 163kB 2.8MB/s 


In [ ]:
# Download the Stanford CoreNLP Java library and unzip it to a ./corenlp folder
!echo "Downloading CoreNLP..."
!wget "http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip" -O corenlp.zip
!unzip corenlp.zip
!mv ./stanford-corenlp-full-2018-10-05 ./corenlp



--2019-12-11 03:10:06--  http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip [following]
--2019-12-11 03:10:07--  https://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 393239982 (375M) [application/zip]
Saving to: ‘corenlp.zip’

corenlp.zip         100%[===================>] 375.02M  13.8MB/s    in 28s     

2019-12-11 03:10:35 (13.5 MB/s) - ‘corenlp.zip’ saved [393239982/393239982]

Archive:  corenlp.zip
   creating: stanford-corenlp-full-2018-10-05/
  inflating: stanford-corenlp-full-2018-10-05/jaxb-core-2.3.0.1-sources.jar  
  inflating: stan

In [ ]:
# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = "./corenlp"

In [ ]:
# Import client module
from stanfordnlp.server import CoreNLPClient

In [ ]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(properties={'annotators': 'coref', 'coref.algorithm' : 'statistical'},annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner','coref'], memory='4G', endpoint='http://localhost:9001',timeout=10000)
print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)

Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 10000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-7227874323f8435c.props -preload tokenize,ssplit,pos,lemma,ner,coref


In [ ]:
#text = "Barack Obama was not born in Hawaii.He was born in Newyork."
text="The system should be able to return a product before processing the credentials of the user \
 who is placing the order on the same day that the order is placed."
#text="The musical store receives tape requests from customers."
#text="System verifies login information for candidate"
# set up the client
#with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'depparse'], timeout=60000, memory='16G') as client:
# submit the request to the server
ann = client.annotate(text)

offset = 0 # keeps track of token offset for each sentence
for sentence in ann.sentence:
    print('___________________')
    print('dependency parse:')
    # extract dependency parse
    dp = sentence.basicDependencies
    # build a helper dict to associate token index and label
    token_dict = {sentence.token[i].tokenEndIndex-offset : sentence.token[i].word for i in range(0, len(sentence.token))}
    offset += len(sentence.token)

    # build list of (source, target) pairs
    out_parse = [(dp.edge[i].source, dp.edge[i].target) for i in range(0, len(dp.edge))]

    for source, target in out_parse:
        print(source, token_dict[source], '->', target, token_dict[target])

    print('\nTokens \t POS \t NER')
    for token in sentence.token:
        print (token.word, '\t', token.pos, '\t', token.ner)


___________________
dependency parse:
2 system -> 1 The
5 able -> 2 system
5 able -> 3 should
5 able -> 4 be
5 able -> 7 return
5 able -> 31 .
7 return -> 6 to
7 return -> 9 product
7 return -> 11 processing
9 product -> 8 a
11 processing -> 10 before
11 processing -> 13 credentials
13 credentials -> 16 user
13 credentials -> 12 the
16 user -> 19 placing
16 user -> 14 of
16 user -> 15 the
19 placing -> 17 who
19 placing -> 18 is
19 placing -> 21 order
19 placing -> 25 day
19 placing -> 30 placed
21 order -> 20 the
25 day -> 22 on
25 day -> 23 the
25 day -> 24 same
28 order -> 27 the
30 placed -> 26 that
30 placed -> 28 order
30 placed -> 29 is

Tokens 	 POS 	 NER
The 	 DT 	 O
system 	 NN 	 O
should 	 MD 	 O
be 	 VB 	 O
able 	 JJ 	 O
to 	 TO 	 O
return 	 VB 	 O
a 	 DT 	 O
product 	 NN 	 O
before 	 IN 	 O
processing 	 VBG 	 O
the 	 DT 	 O
credentials 	 NNS 	 O
of 	 IN 	 O
the 	 DT 	 O
user 	 NN 	 O
who 	 WP 	 O
is 	 VBZ 	 O
placing 	 VBG 	 O
the 	 DT 	 O
order 	 NN 	 O
on 	 IN 	 O
the 	 

In [ ]:
#def depparse(text):
#text="Chris Manning is a nice person. Chris wrote a simple sentence. He also gives oranges to people."
#text = "Barack Obama was not born in Hawaii. He was born in Newyork."
parsed=""
nlp=client

output = nlp.annotate(text, properties={
'annotators': 'tokenize,ssplit,pos,depparse,parse',
"timeout": "50000",
'outputFormat': 'json'

 })
for i in output['sentences']: # not sure if there can be multiple items here. If so, it just returns the first one currently.
  for dep in i['basicDependencies']:
    print(dep['dep'],dep['governorGloss'],dep['governor'])
  #print( [tuple((dep['dep'], dep['governorGloss'], dep['dependentGloss'])) for dep in i['basicDependencies']])
print(output['sentences'][0]['parse'])
print('------------------------------')
#print(output['sentences'][1]['parse'])

ROOT ROOT 0
det system 2
nsubj able 5
aux able 5
cop able 5
mark return 7
xcomp able 5
det product 9
dobj return 7
mark processing 11
advcl return 7
det credentials 13
dobj processing 11
case user 16
det user 16
nmod credentials 13
nsubj placing 19
aux placing 19
acl:relcl user 16
det order 21
dobj placing 19
case day 25
det day 25
amod day 25
nmod order 21
mark placed 30
det order 28
nsubjpass placed 30
auxpass placed 30
ccomp able 5
punct able 5
(ROOT
  (S
    (NP (DT The) (NN system))
    (VP (MD should)
      (VP (VB be)
        (ADJP (JJ able)
          (S
            (VP (TO to)
              (VP (VB return)
                (NP (DT a) (NN product))
                (PP (IN before)
                  (S
                    (VP (VBG processing)
                      (NP
                        (NP (DT the) (NNS credentials))
                        (PP (IN of)
                          (NP
                            (NP (DT the) (NN user))
                            (SBAR
         

In [ ]:
nlp=client
output = nlp.semgrex(text, pattern='{tag: VBN}', filter=False)
print(output)

{'sentences': [{'0': {'text': 'placed', 'begin': 29, 'end': 30}, 'length': 1}]}


In [ ]:
#text="System verifies login information for user."
text="The musical store receives tape requests from customers. The musical store receives new tapes from the Main office."
output = nlp.annotate(
        text, properties={"annotators": "tokenize,ssplit,pos,depparse,parse,ner", "outputFormat": "json"}
)
# pprint.pprint(output)
#tree = output["sentences"][0]["parse"]
#print(tree)
l=len(output["sentences"])
for j in range(l):
  x = output["sentences"][j]["basicDependencies"]
  # pprint.pprint(x)
  print ("-------------------------------------------------")
  for i in range(len(x)):
      print (x[i]["dep"] + "-->" + x[i]["governorGloss"] + "-" + str(x[i]["governor"]) + " " + x[i][
          "dependentGloss"
      ] + "-" + str(x[i]["dependent"]))
  for i in range(len(x)):
    if x[i]["dep"]=='ROOT':
      msg=str(x[i]["dependentGloss"])
    elif x[i]["dep"]=='nsubj':
      send=str(x[i]["dependentGloss"])
    elif x[i]["dep"]=='dobj':
      msg1=str(x[i]["dependentGloss"])
    elif x[i]["dep"]=='nmod':
      rec=str(x[i]["dependentGloss"])
  print("-------------------------------")
  print (send + "-->" + msg+ " "+msg1+"-->" + rec) 

-------------------------------------------------
ROOT-->ROOT-0 receives-4
det-->store-3 The-1
amod-->store-3 musical-2
nsubj-->receives-4 store-3
compound-->requests-6 tape-5
dobj-->receives-4 requests-6
case-->customers-8 from-7
nmod-->receives-4 customers-8
punct-->receives-4 .-9
-------------------------------
store-->receives requests-->customers
-------------------------------------------------
ROOT-->ROOT-0 receives-4
det-->store-3 The-1
amod-->store-3 musical-2
nsubj-->receives-4 store-3
amod-->tapes-6 new-5
dobj-->receives-4 tapes-6
case-->office-10 from-7
det-->office-10 the-8
compound-->office-10 Main-9
nmod-->receives-4 office-10
punct-->receives-4 .-11
-------------------------------
store-->receives tapes-->office


In [ ]:
def typedependencies(sent_list, neg_words, compound_word_list):

    pos_dict = {}
    depend_dict = {}
    depend_list = []
    proper_names = []
    # neg_words = []
    compound_dic = {}

    #nlp = StanfordCoreNLP("http://localhost:9000")
    nlp=client
    for i in range(len(sent_list)):
        compound_list = []
        print(sent_list[i])
        output = nlp.annotate(
            sent_list[i], properties={"annotators": "tokenize,ssplit,pos,depparse,parse,ner", "outputFormat": "json"}
        )
        # pprint.pprint(output)
        x = output["sentences"][0]["basicDependencies"]
        # pprint.pprint(output['sentences'][0]['parse'])
        # pprint.pprint(x)
        # print '-------------------------------------------------'
        for j in range(len(x)):

            if "compound" in x[j]["dep"]:
                # compound_word(x[j])
                ll = [x[j]["governorGloss"], x[j]["governor"], x[j]["dependentGloss"], x[j]["dependent"]]
                compound_dic[x[j]["governor"]] = x[j]["governorGloss"]
                compound_dic[x[j]["dependent"]] = x[j]["dependentGloss"]
                # compound_list.append(ll)

            d = [
                x[j]["dep"],
                x[j]["governorGloss"],
                str(x[j]["governor"]),
                x[j]["dependentGloss"],
                str(x[j]["dependent"]),
            ]
            depend_list.append(d)

            # getting the negative words..
            if "neg" in x[j]["dep"]:
                x1 = x[j]["governorGloss"].lower()
                x2 = x[j]["dependentGloss"].lower()
                if x1 not in stopwords:
                    neg_words.append([x1, x[j]["governor"]])
                else:
                    neg_words.append([x2, x[j]["dependent"]])

            if "conj" in x[j]["dep"]:
                x1 = x[j]["governorGloss"].lower()
                x2 = x[j]["dependentGloss"].lower()
                if x1 in neg_prefix:
                    neg_words.append([x2, x[j]["dependent"]])
                # elif (x2 == 'not' or x2 == 'nor' or x2 == 'non'):
                #   neg_words.append(x1)
                elif x2 in neg_prefix:
                    neg_words.append([x1, x[j]["governor"]])

            print (
                x[j]["dep"]
                + "-->"
                + x[j]["governorGloss"]
                + "-"
                + str(x[j]["governor"])
                + " "
                + x[j]["dependentGloss"]
                + "-"
                + str(x[j]["dependent"])
            )
        print ("===================================")

        for key, value in sorted(compound_dic.items()):
            compound_list.append([key, value])
        # print compound_word(compound_list)
        compound_dic.clear()

        y = output["sentences"][0]["tokens"]
        for k in range(len(y)):
            pos_dict[y[k]["word"]] = y[k]["pos"]
            if "NNP" in y[k]["pos"]:
                proper_names.append(y[k]["word"])

        depend_dict[i] = depend_list
        depend_list = []

        if len(compound_list) > 0:
            w = compound_word(compound_list)
        else:
            w = []
        for jj in range(len(w)):
            if w[jj] != "":
                print (w[jj])
                compound_word_list.append(w[jj])

    print ("--------NAMES------" + str(proper_names))
    print ("--------NEGATIVE----" + str(neg_words))
    return depend_dict, pos_dict, proper_names


In [ ]:

text =list("Barack Obama was not born in Hawaii. He was born in Newyork.")
typedependencies(text, "", "")

B
ROOT-->ROOT-0 B-1
a
ROOT-->ROOT-0 a-1
r
ROOT-->ROOT-0 r-1
a
ROOT-->ROOT-0 a-1
c
ROOT-->ROOT-0 c-1
k
ROOT-->ROOT-0 k-1
 


IndexError: ignored

In [ ]:
nlp=client
text="The musical store receives tape requests from customers."
output = nlp.semgrex(text, pattern='{tag: VBZ}', filter=False)
print(output)

{'sentences': [True]}


In [ ]:
text = "Barack Obama was not born in Hawaii. He was born in Newyork."
#text="The musical store receives tape requests from customers."
#text="System verifies login information for candidate"
# set up the client
#with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'depparse'], timeout=60000, memory='16G') as client:
# submit the request to the server
ann = client.annotate(text)
# You can access annotations using ann.
sentence = ann.sentence[0]

# The corenlp.to_text function is a helper function that
# reconstructs a sentence from tokens.

#print(ann.text)
# You can access any property within a sentence.
for sentence in ann.sentence:
  for token in sentence.token:
    print(token.word,token.pos,token.ner)

Barack NNP PERSON
Obama NNP PERSON
was VBD O
not RB O
born VBN O
in IN O
Hawaii NNP STATE_OR_PROVINCE
. . O
He PRP O
was VBD O
born VBN O
in IN O
Newyork NNP LOCATION
. . O


In [ ]:
#def depparse(text):
text="Chris Manning is a nice person. Chris wrote a simple sentence. He also gives oranges to people."

parsed=""
nlp=client

output = nlp.annotate(text, properties={
  'annotators': 'depparse',
  'outputFormat': 'json'
  })
for i in output['sentences']: # not sure if there can be multiple items here. If so, it just returns the first one currently.
  print( [tuple((dep['dep'], dep['governorGloss'], dep['dependentGloss'])) for dep in i['basicDependencies']])

[('ROOT', 'ROOT', 'person'), ('compound', 'Manning', 'Chris'), ('nsubj', 'person', 'Manning'), ('cop', 'person', 'is'), ('det', 'person', 'a'), ('amod', 'person', 'nice'), ('punct', 'person', '.')]
[('ROOT', 'ROOT', 'wrote'), ('nsubj', 'wrote', 'Chris'), ('det', 'sentence', 'a'), ('amod', 'sentence', 'simple'), ('dobj', 'wrote', 'sentence'), ('punct', 'wrote', '.')]
[('ROOT', 'ROOT', 'gives'), ('nsubj', 'gives', 'He'), ('advmod', 'gives', 'also'), ('dobj', 'gives', 'oranges'), ('case', 'people', 'to'), ('nmod', 'gives', 'people'), ('punct', 'gives', '.')]


In [ ]:
import re 
import string 
import nltk 
import spacy 
import pandas as pd 
import numpy as np 
import math 
from tqdm import tqdm 

from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 

pd.set_option('display.max_colwidth', 200)

In [ ]:
# load spaCy model
nlp = spacy.load("en_core_web_sm")

In [ ]:
#2. Split compund sentence
def split_sentence(consent):
  simsent=list()
  sent=""
  d = nlp(consent)
  for tok in d:
    #print(tok.text)
    if tok.text in ('AND','OR','and','or',';'):
      simsent.append(sent)
      print(sent)
      sent=""
    else:
      sent=sent+" "+tok.text
  simsent.append(sent)
  return simsent


In [ ]:
sent=split_sentence("He is good and he is old")
for s in sent:
  print(s)

 He is good
 He is good
 he is old


In [ ]:
nlp=client
text="System verifies login information for candidate"
text = "Chris wrote a simple sentence. he parsed with Stanford CoreNLP."
output = nlp.annotate(text, properties={"annotators":"tokenize,ssplit,pos,depparse,natlog,openie",
                            "outputFormat": "json",
                             "openie.triple.strict":"true",
                             "openie.max_entailments_per_clause":"1"})
result = [output["sentences"][0]["openie"] for item in output]
for i in result:
    #print(i)
    for rel in i:
        relationSent=rel['subject'],rel['relation'],rel['object']
        #S=rel['subject']
        #O=rel['object']
        #A=rel['relation']
        #print(S,A,O)
        print(relationSent)

In [ ]:
text = "Chris wrote a simple sentence that he parsed with Stanford CoreNLP."

ann = client.annotate(text)

# You can access annotations using ann.
sentence = ann.sentence[0]
# Likewise for tokens
token = sentence.token[0]
print(token.lemma)

# Use tokensregex patterns to find who wrote a sentence.
pattern = '([ner: PERSON]+) /wrote/ /an?/ []{0,3} /sentence|article/'
matches = client.tokensregex(text, pattern)
print(matches)

KeyboardInterrupt: ignored

In [ ]:
# Use semgrex patterns to directly find who wrote what.
pattern = '{word:wrote} >nsubj {}=subject >dobj {}=object'
matches = client.semgrex(text, pattern)

#print(matches["sentences"][1]["0"]["text"])
#print(matches["sentences"][1]["0"]["$subject"]["text"] )
#print(matches["sentences"][1]["0"]["$object"]["text"] )

In [ ]:
nlp=client
text = 'Mark Robert is the founder of 3trucks. 3trucks was founded in 2010'

output = nlp.annotate(text, properties={
'annotators': 'tokenize,ssplit,pos,depparse,parse',
"timeout": "50000",
'outputFormat': 'json'

 })

print(output['sentences'][0]['parse'])
print('------------------------------')
print(output['sentences'][1]['parse'])

(ROOT
  (S
    (NP (NNP Mark) (NNP Robert))
    (VP (VBZ is)
      (NP
        (NP (DT the) (NN founder))
        (PP (IN of)
          (NP (NNS 3trucks)))))
    (. .)))
------------------------------
(ROOT
  (S
    (NP (NNS 3trucks))
    (VP (VBD was)
      (VP (VBN founded)
        (PP (IN in)
          (NP (CD 2010)))))))


In [ ]:
text="The musical store receives tape requests from customers. The musical store receives \\
new tapes from the Main office. Musical store sends overdue notice to customers. \\
Store assistant takes care of tape requests. Store assistant update the rental list. Store \\
management submits the price changes. Store management submits new tapes. Store \\
administration produces rental reports. Main office sends overdue notices for tapes. \\
Customer request for a tape. Store assistant checks the availability of requested tape.\\
Store assistant searches for the available tape. Store assistant searches for the rental \\
price of available tape. Store assistant checks status of the tape to be returned by \\
customer. Customer can borrow if there is no delay with return of other tapes. Store \\
assistant records rental by updating the rental list. Store assistant asks customer for \\
his address."

In [ ]:
output = client.annotate(text, properties={'annotators':'dcoref','outputFormat':'json','ner.useSUTime':'false'})

coreferences = output['corefs']
print(coreferences)
print(document.hasCorefAnnotation)

ReadTimeout: ignored

In [ ]:
#Sample data
sam1="User needed money for fees. He went to the ATM. He \
entered password into the machine. User put the \
money in his pocket."
sam2="The Person walks over to the ATM. ATM asks\
password from the user. The user enter password \
into the machine."
sample="Each lift has a set of buttons, one for each floor. These illuminate when pressed and cause the lift to visit the corresponding floor. The illumination is cancelled when the corresponding floor is visited by the lift."""

In [ ]:
#from pycorenlp import StanfordCoreNLP
#Solving coreferences
nlp = client #StanfordCoreNLP('http://localhost:9000')


def resolve(corenlp_output):
    """ Transfer the word form of the antecedent to its associated pronominal anaphor(s) """
    for coref in corenlp_output['corefs']:
        mentions = corenlp_output['corefs'][coref]
        antecedent = mentions[0]  # the antecedent is the first mention in the coreference chain
        for j in range(1, len(mentions)):
            mention = mentions[j]
            if mention['type'] == 'PRONOMINAL':
                # get the attributes of the target mention in the corresponding sentence
                target_sentence = mention['sentNum']
                target_token = mention['startIndex'] - 1
                # transfer the antecedent's word form to the appropriate token in the sentence
                corenlp_output['sentences'][target_sentence - 1]['tokens'][target_token]['word'] = antecedent['text']


def print_resolved(corenlp_output):
    """ Print the "resolved" output """
    possessives = ['hers', 'his', 'their', 'theirs']
    for sentence in corenlp_output['sentences']:
        for token in sentence['tokens']:
            output_word = token['word']
            # check lemmas as well as tags for possessive pronouns in case of tagging errors
            if token['lemma'] in possessives or token['pos'] == 'PRP$':
                output_word += "'s"  # add the possessive morpheme
            output_word += token['after']
            print(output_word, end='')


text ="Tom and Jane are good friends. They are cool. He knows a lot of things and so does she. His car is red, but " \
       "hers is blue. It is older than hers. The big cat ate its dinner."

output = nlp.annotate(text, properties= {'annotators':'dcoref','outputFormat':'json','ner.useSUTime':'false'})

resolve(output)

print('Original:', text)
print('Resolved: ', end='')
print_resolved(output)

ReadTimeout: ignored

In [ ]:
#onverting complex sentence to simple
import re

text = """\
Mr. Smith bought cheapsite.com or 1.5 million dollars, and i.e. he paid a lot for it. Did he mind? Adam Jones Jr. thinks he didn't. In any case, this isn't true... Well, with a probability of .9 it isn't.
"""
#sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)
sentences = re.split(r'(and|or)', text)
for stuff in sentences:
        print(stuff)
re.sub('(a|an|and|the|is)', "", text)

Mr. Smith bought cheapsite.com 
or
 1.5 million dollars, 
and
 i.e. he paid a lot f
or
 it. Did he mind? Adam Jones Jr. thinks he didn't. In any case, this isn't true... Well, with a probability of .9 it isn't.



"Mr. Smith bought chepsite.com or 1.5 million dollrs, nd i.e. he pid  lot for it. Did he mind? Adm Jones Jr. thinks he didn't. In ny cse, th n't true... Well, with  probbility of .9 it n't.\n"

In [ ]:
import nltk
from nltk import Tree
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")
#from nltk import groucho_grammar

sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
parser = nltk.ChartParser(groucho_grammar)
for tree in parser.parse(sent):
  print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


In [ ]:
#from stanfordnlp.server import CoreNLPClient
from nltk import tokenize
#nltk.download('punkt')
client = client
def pronoun_resolution(text):

    ann = client.annotate(text)
    modified_text = tokenize.sent_tokenize(text)

    for coref in ann.corefChain:

        antecedent = []
        for mention in coref.mention:
            phrase = []
            for i in range(mention.beginIndex, mention.endIndex):
                phrase.append(ann.sentence[mention.sentenceIndex].token[i].word)
            if antecedent == []:
                antecedent = ' '.join(word for word in phrase)
            else:
                anaphor = ' '.join(word for word in phrase)
                modified_text[mention.sentenceIndex] = modified_text[mention.sentenceIndex].replace(anaphor, antecedent)

    modified_text = ' '.join(modified_text)

    return modified_text

text = 'Tom is a smart boy. He knows a lot of things.'
print(pronoun_resolution(text))

Tom is a smart boy. He knows a lot of things.


In [ ]:
from stanfordnlp.server import CoreNLPClient

text = 'Barack was born in Hawaii. His wife Michelle was born in Milan. He says that she is very smart.'
print(f"Input text: {text}")

# set up the client
client = CoreNLPClient(properties={'annotators': 'coref', 'coref.algorithm' : 'statistical'}, timeout=60000, memory='16G')

# submit the request to the server
ann = client.annotate(text)    

mychains = list()
print(ann.hasCorefAnnotation)
chains = ann.corefChain
print(chains)
for chain in chains:
    mychain = list()
    # Loop through every mention of this chain
    for mention in chain.mention:
        # Get the sentence in which this mention is located, and get the words which are part of this mention
        # (we can have more than one word, for example, a mention can be a pronoun like "he", but also a compound noun like "His wife Michelle")
        words_list = ann.sentence[mention.sentenceIndex].token[mention.beginIndex:mention.endIndex]
        #build a string out of the words of this mention
        ment_word = ' '.join([x.word for x in words_list])
        mychain.append(ment_word)
    mychains.append(mychain)

for chain in mychains:
    print(' <-> '.join(chain))
#edited Jul 19 at 10:48


Input text: Barack was born in Hawaii. His wife Michelle was born in Milan. He says that she is very smart.
Starting server with command: java -Xmx16G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-9ae256c370644b19.props -preload coref


PermanentlyFailedException: ignored

In [ ]:


from nltk.parse.stanford import StanfordDependencyParser
sdp = StanfordDependencyParser(path_to_jar='E:/stanford/stanford-parser-full-2015-04-20/stanford-parser.jar',
                               path_to_models_jar='E:/stanford/stanford-parser-full-2015-04-20/stanford-parser-3.5.2-models.jar')    

result = list(sdp.raw_parse(sentence))  

# print the dependency tree
dep_tree = [parse.tree() for parse in result][0]
print(dep_tree)

# visualize raw dependency tree
from IPython.display import display
display(dep_tree)

# visualize annotated dependency tree (needs graphviz)
from graphviz import Source
dep_tree_dot_repr = [parse for parse in result][0].to_dot()
source = Source(dep_tree_dot_repr, filename="dep_tree", format="png")
source

LookupError: ignored

https://medium.com/analytics-vidhya/introduction-to-information-extraction-using-python-and-spacy-858f5d6416ca

In [ ]:
import re 
import string 
import nltk 
import spacy 
import pandas as pd 
import numpy as np 
import math 
from tqdm import tqdm 

from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 

pd.set_option('display.max_colwidth', 200)

In [ ]:
# load spaCy model
nlp = spacy.load("en_core_web_sm")

In [ ]:
# sample text 
text = "GDP in developing countries such as Vietnam will continue growing at a high rate." 

# create a spaCy object 
doc = nlp(text)

In [ ]:
# print token, dependency, POS tag 
for tok in doc: 
  print(tok.text, "-->",tok.dep_,"-->", tok.pos_)

GDP --> nsubj --> NOUN
in --> prep --> ADP
developing --> amod --> VERB
countries --> pobj --> NOUN
such --> amod --> ADJ
as --> prep --> ADP
Vietnam --> pobj --> PROPN
will --> aux --> VERB
continue --> ROOT --> VERB
growing --> xcomp --> VERB
at --> prep --> ADP
a --> det --> DET
high --> amod --> ADJ
rate --> pobj --> NOUN
. --> punct --> PUNCT


In [ ]:
#define the pattern 
pattern = [{'POS':'NOUN'}, 
           {'LOWER': 'such'}, 
           {'LOWER': 'as'}, 
           {'POS': 'PROPN'} ]#proper noun]

In [ ]:
# Matcher class object 
matcher = Matcher(nlp.vocab) 
matcher.add("matching_1", None, pattern) 

matches = matcher(doc) 
span = doc[matches[0][1]:matches[0][2]] 

print(span.text)

countries such as Vietnam


In [ ]:
# Matcher class object
matcher = Matcher(nlp.vocab)

#define the pattern
pattern = [{'DEP':'amod', 'OP':"?"}, # adjectival modifier
           {'POS':'NOUN'},
           {'LOWER': 'such'},
           {'LOWER': 'as'},
           {'POS': 'PROPN'}]

matcher.add("matching_1", None, pattern)
matches = matcher(doc)

span = doc[matches[0][1]:matches[0][2]]
print(span.text)

developing countries such as Vietnam


In [ ]:
######1. find subj obj verb
text="user selects the company for which he wants to apply."
#text="Here is how you can keep your car and other vehicles clean."
doc = nlp(text) 

# print dependency tags and POS tags
for tok in doc: 
  #print(tok.text, "-->",tok.dep_, "-->",tok.pos_)
  print(tok.text,"/",tok.pos_,"/",tok.dep_,"/",tok.tag_,"/",tok.orth_)
for tok in doc:
  if tok.tag_ in ('NN','NP') and tok.dep_ in ('nsubj'):
    sub=tok.text

for tok in doc:
  if tok.tag_ in ('NN','NP') and tok.dep_ in ('dobj'):
    obj=tok.text

for tok in doc:
  if tok.tag_ in ('VBZ') and tok.dep_ in ('ROOT'):
    action=tok.text
print(sub,obj,action)

user / NOUN / nsubj / NN / user
selects / VERB / ROOT / VBZ / selects
the / DET / det / DT / the
company / NOUN / dobj / NN / company
for / ADP / prep / IN / for
which / DET / pobj / WDT / which
he / PRON / nsubj / PRP / he
wants / VERB / relcl / VBZ / wants
to / PART / aux / TO / to
apply / VERB / xcomp / VB / apply
. / PUNCT / punct / . / .
user company selects


In [ ]:
import spacy
nlp = spacy.load('en')
parsed_text = nlp(u"I thought it was the complete set")
direct_object=""
indirect_object=""
#get token dependencies
for text in parsed_text:
    #subject would be
    if text.dep_ == "nsubj":
        subject = text.orth_
    #iobj for indirect object
    if text.dep_ == "iobj":
        indirect_object = text.orth_
    #dobj for direct object
    if text.dep_ == "dobj":
        direct_object = text.orth_

print(subject)
print(direct_object)
print(indirect_object)

it




In [ ]:
doc = nlp("I thought it was the complete set")
for nc in doc.noun_chunks:
    print(nc.text)

I
it
the complete set


In [ ]:
import spacy
nlp = spacy.load('en')
doc = nlp(u'KEEP CALM because TOGETHER We Rock !')
for word in doc:
    print(word.text, word.lemma, word.lemma_, word.tag, word.tag_, word.pos, word.pos_)
    print(word.orth_)

KEEP 10409167902684988128 KEEP 15794550382381185553 NNP 96 PROPN
KEEP
CALM 18347787977573315042 CALM 15794550382381185553 NNP 96 PROPN
CALM
because 16950148841647037698 because 1292078113972184607 IN 85 ADP
because
TOGETHER 13495226875806840535 TOGETHER 15794550382381185553 NNP 96 PROPN
TOGETHER
We 561228191312463089 -PRON- 13656873538139661788 PRP 95 PRON
We
Rock 3237817430745561104 Rock 15794550382381185553 NNP 96 PROPN
Rock
! 17494803046312582752 ! 12646065887601541794 . 97 PUNCT
!


In [ ]:
# Matcher class object 
matcher = Matcher(nlp.vocab) 

#define the pattern 
pattern = [{'DEP':'amod', 'OP':"?"}, 
           {'POS':'NOUN'}, 
           {'LOWER': 'and', 'OP':"?"}, 
           {'LOWER': 'or', 'OP':"?"}, 
           {'LOWER': 'other'}, 
           {'POS': 'NOUN'}] 
           
matcher.add("matching_1", None, pattern) 

matches = matcher(doc) 
span = doc[matches[0][1]:matches[0][2]] 
print(span.text)

car and other vehicles


In [ ]:
# replaced 'and' with 'or' 
doc = nlp("Here is how you can keep your car or other vehicles clean.")
# Matcher class object 
matcher = Matcher(nlp.vocab) 

#define the pattern 
pattern = [{'DEP':'amod', 'OP':"?"}, 
           {'POS':'NOUN'}, 
           {'LOWER': 'and', 'OP':"?"}, 
           {'LOWER': 'or', 'OP':"?"}, 
           {'LOWER': 'other'}, 
           {'POS': 'NOUN'}] 
           
matcher.add("matching_1", None, pattern) 

matches = matcher(doc) 
span = doc[matches[0][1]:matches[0][2]] 
print(span.text)

car or other vehicles


In [ ]:
doc = nlp("A healthy eating pattern including fruits, including whole fruits.") 

# Matcher class object 
matcher = Matcher(nlp.vocab) 

#define the pattern 
pattern = [{'DEP':'nummod','OP':"?"}, # numeric modifier 
           {'DEP':'amod','OP':"?"}, # adjectival modifier 
           {'POS':'NOUN'}, 
           {'IS_PUNCT': True}, 
           {'LOWER': 'including'}, 
           {'DEP':'nummod','OP':"?"}, 
           {'DEP':'amod','OP':"?"}, 
           {'POS':'NOUN'}] 
                               
matcher.add("matching_1", None, pattern) 

matches = matcher(doc) 
span = doc[matches[0][1]:matches[0][2]] 
print(span.text)

fruits, including whole fruits


In [ ]:
doc = nlp("A healthy eating pattern includes fruits, especially whole fruits.") 

for tok in doc: 
  print(tok.text, tok.dep_, tok.pos_)

A det DET
healthy amod ADJ
eating compound NOUN
pattern nsubj NOUN
includes ROOT VERB
fruits dobj NOUN
, punct PUNCT
especially advmod ADV
whole amod ADJ
fruits appos NOUN
. punct PUNCT


In [ ]:
# Matcher class object 
matcher = Matcher(nlp.vocab)

#define the pattern 
pattern = [{'DEP':'nummod','OP':"?"}, 
           {'DEP':'amod','OP':"?"}, 
           {'POS':'NOUN'}, 
           {'IS_PUNCT':True}, 
           {'LOWER': 'especially'}, 
           {'DEP':'nummod','OP':"?"}, 
           {'DEP':'amod','OP':"?"}, 
           {'POS':'NOUN'}] 
           
matcher.add("matching_1", None, pattern) 

matches = matcher(doc) 
span = doc[matches[0][1]:matches[0][2]] 
print(span.text)

fruits, especially whole fruits


In [ ]:
#dependency
text = "Tableau was recently acquired by Salesforce." 

# Plot the dependency graph 
doc = nlp(text) 
displacy.render(doc, style='dep',jupyter=True)


In [ ]:
text='“Careem, a ride-hailing major in the middle east, was acquired by Uber.”'
# Plot the dependency graph 
doc = nlp(text) 
displacy.render(doc, style='dep',jupyter=True)


In [ ]:
#Subtree matching
text = "Tableau was recently acquired by Salesforce." 
doc = nlp(text) 

for tok in doc: 
  print(tok.text,"-->",tok.dep_,"-->",tok.pos_)


Tableau --> nsubjpass --> PROPN
was --> auxpass --> VERB
recently --> advmod --> ADV
acquired --> ROOT --> VERB
by --> agent --> ADP
Salesforce --> pobj --> PROPN
. --> punct --> PUNCT


In [ ]:
def subtree_matcher(doc): 
  x = '' 
  y = '' 
  
  # iterate through all the tokens in the input sentence 
  for i,tok in enumerate(doc): 
    # extract subject 
    if tok.dep_.find("subjpass") == True: 
      y = tok.text 
      
    # extract object 
    if tok.dep_.endswith("obj") == True: 
      x = tok.text 
      
  return x,y

In [ ]:
subtree_matcher(doc)

('Salesforce', 'Tableau')

In [ ]:
text_2 = "Careem, a ride hailing major in middle east, was acquired by Uber." 

doc_2 = nlp(text_2) 
subtree_matcher(doc_2)

('Uber', 'Careem')

In [ ]:
#Using active sentence instead of passiv
text_3 = "Salesforce recently acquired Tableau." 
doc_3 = nlp(text_3) 
subtree_matcher(doc_3)

('Tableau', '')

In [ ]:
#Sub tree matching for both active and passive sentences
def new_subtree_matcher(doc):
  subjpass = 0

  for i,tok in enumerate(doc):
    # find dependency tag that contains the text "subjpass"    
    if tok.dep_.find("subjpass") == True:
      subjpass = 1

  x = ''
  y = ''

  # if subjpass == 1 then sentence is passive
  if subjpass == 1:
    for i,tok in enumerate(doc):
      if tok.dep_.find("subjpass") == True:
        y = tok.text

      if tok.dep_.endswith("obj") == True:
        x = tok.text
  
  # if subjpass == 0 then sentence is not passive
  else:
    for i,tok in enumerate(doc):
      if tok.dep_.endswith("subj") == True:
        x = tok.text

      if tok.dep_.endswith("obj") == True:
        y = tok.text

  return x,y

In [ ]:
new_subtree_matcher(doc_3)

('Salesforce', 'Tableau')

In [ ]:
new_subtree_matcher(nlp("Tableau was recently acquired by Salesforce."))

('Salesforce', 'Tableau')

In [ ]:
text="System verifies login information of candidate"
doc=nlp(text)
# print dependency tags and POS tags
for tok in doc: 
  print(tok.text, "-->",tok.dep_, "-->",tok.pos_)


System --> compound --> NOUN
verifies --> nsubj --> NOUN
login --> ROOT --> VERB
information --> dobj --> NOUN
of --> prep --> ADP
candidate --> pobj --> NOUN


In [ ]:
doc = nlp(
        "displaCy uses CSS and JavaScript to show you how computers "
        "understand language"
    )

# The easiest way is to find the head of the subtree you want, and then use
# the `.subtree`, `.children`, `.lefts` and `.rights` iterators. `.subtree`
# is the one that does what you're asking for most directly:
for word in doc:
    if word.dep_ in ("xcomp", "ccomp"):
        print("".join(w.text_with_ws for w in word.subtree))

to show you how computers understand language
how computers understand language


In [ ]:
# It'd probably be better for `word.subtree` to return a `Span` object
# instead of a generator over the tokens. If you want the `Span` you can
# get it via the `.right_edge` and `.left_edge` properties. The `Span`
# object is nice because you can easily get a vector, merge it, etc.
for word in doc:
    if word.dep_ in ("xcomp", "ccomp"):
        subtree_span = doc[word.left_edge.i : word.right_edge.i + 1]
        print(subtree_span.text, "|", subtree_span.root.text)


to show you how computers understand language | show
how computers understand language | understand


In [ ]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
pattern = [{"LOWER": "hello"}, {"IS_PUNCT": True}, {"LOWER": "world"}]
matcher.add("HelloWorld", None, pattern)

doc = nlp("Hello, world! Hello world!")
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

15578876784678163569 HelloWorld 0 3 Hello, world


In [ ]:
matcher.add("HelloWorld", None,
            [{"LOWER": "hello"}, {"IS_PUNCT": True}, {"LOWER": "world"}],
            [{"LOWER": "hello"}, {"LOWER": "world"}])

In [ ]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
matched_sents = []  # Collect data of matched sentences to be visualized

def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]  # Matched span
    sent = span.sent  # Sentence containing matched span
    # Append mock entity for match in displaCy style to matched_sents
    # get the match span by ofsetting the start and end of the span with the
    # start and end of the sentence in the doc
    match_ents = [{
        "start": span.start_char - sent.start_char,
        "end": span.end_char - sent.start_char,
        "label": "MATCH",
    }]
    matched_sents.append({"text": sent.text, "ents": match_ents})

pattern = [{"LOWER": "facebook"}, {"LEMMA": "be"}, {"POS": "ADV", "OP": "*"},
           {"POS": "ADJ"}]
matcher.add("FacebookIs", collect_sents, pattern)  # add pattern
doc = nlp("I'd say that Facebook is evil. – Facebook is pretty cool, right?")
matches = matcher(doc)

# Serve visualization of sentences containing match with displaCy
# set manual=True to make displaCy render straight from a dictionary
# (if you're not running the code within a Jupyer environment, you can
# use displacy.serve instead)
displacy.render(matched_sents, style="ent", manual=True)

'<div class="entities" style="line-height: 2.5; direction: ltr">I\'d say that \n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone">\n    Facebook is evil\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">MATCH</span>\n</mark>\n.</div>\n\n<div class="entities" style="line-height: 2.5; direction: ltr">\n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone">\n    Facebook is pretty cool\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">MATCH</span>\n</mark>\n, right?</div>'

In [ ]:
import spacy
import networkx as nx
nlp = spacy.load("en_core_web_sm")
doc = nlp(u'Convulsions that occur after DTaP are caused by a fever.')

print('sentence:'.format(doc))

# Load spacy's dependency tree into a networkx graph
edges = []
for token in doc:
    for child in token.children:
        edges.append(('{0}'.format(token.lower_),
                      '{0}'.format(child.lower_)))
graph = nx.Graph(edges)
# Get the length and path
entity1 = 'Convulsions'.lower()
entity2 = 'fever'
print(nx.shortest_path_length(graph, source=entity1, target=entity2))
print(nx.shortest_path(graph, source=entity1, target=entity2))

sentence:
3
['convulsions', 'caused', 'by', 'fever']


In [ ]:
from nltk.parse.stanford import StanfordParser
from nltk.tree import ParentedTree, Tree
 
parser = StanfordParser()
 
# Parse the example sentence
sent = 'A rare black squirrel has become a regular visitor to a suburban garden'
t = list(parser.raw_parse(sent))[0]
t = ParentedTree.convert(t)
 
t.pretty_print()

LookupError: ignored

In [ ]:
import os
 
os.environ['STANFORD_PARSER'] = 'stanford-parser'
os.environ['STANFORD_MODELS'] = 'stanford-parser'
 

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install StanfordParser

ERROR: Could not find a version that satisfies the requirement StanfordParser (from versions: none)
ERROR: No matching distribution found for StanfordParser


In [ ]:
# Update / Install NLTK
!pip install -U nltk

# Download the Stanford NLP tools
!wget http://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip
!wget http://nlp.stanford.edu/software/stanford-postagger-full-2015-04-20.zip
!wget http://nlp.stanford.edu/software/stanford-parser-full-2015-04-20.zip
# Extract the zip file.
!unzip stanford-ner-2015-04-20.zip 
!unzip stanford-parser-full-2015-04-20.zip 
!unzip stanford-postagger-full-2015-04-20.zip

     |████████████████████████████████| 1.5MB 2.8MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449908 sha256=8a1c0249b807f0ceb12cf75972457d091bae1c006af92abb22bb9c763edb091f
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


--2019-11-26 04:37:59--  http://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip [following]
--2019-11-26 04:37:59--  https://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176961718 (169M) [application/zip]
Saving to: ‘stanford-ner-2015-04-20.zip’

stanford-ner-2015-0 100%[===================>] 168.76M  1.47MB/s    in 2m 13s  

2019-11-26 04:40:12 (1.27 MB/s) - ‘stanford-ner-2015-04-20.zip’ saved [176961718/176961718]

--2019-11-26 04:40:13--  http://nlp.stanford.edu/software/stanford-postagger-full-2015-04-20.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140

In [ ]:
!mv ./stanford-ner-2015-04-20 ./stanford
!mv ./stanford-parser-full-2015-04-20 ./stanford
!mv ./stanford-postagger-full-2015-04-20 ./stanford


In [ ]:
# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["'STANFORD_PARSER'"] = "./stanford/jars"
os.environ['STANFORD_MODELS'] = './standford/jars'

In [ ]:
!pip install textacy

     |████████████████████████████████| 204kB 2.8MB/s 
     |████████████████████████████████| 3.0MB 42.6MB/s 
     |████████████████████████████████| 143kB 46.3MB/s 
     |████████████████████████████████| 481kB 38.6MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.7.2-cp36-cp36m-linux_x86_64.whl size=73004 sha256=50da9a377af9c276c76d174628d9c8482b5825e329f396ff63384ab0edb97fc0
  Stored in directory: /root/.cache/pip/wheels/e8/fe/99/d8fa8f2ef7b82a625b0b77a84d319b0b50693659823c4effb4
  Created wheel for cytoolz: filename=cytoolz-0.10.1-cp36-cp36m-linux_x86_64.whl size=1256647 sha256=3d711e5ac0875ea43d186fb5cf93fdad38f679556f9410490afc820198257a49
  Stored in directory: /root/.cache/pip/wheels/64/2a/18/d962b614e055577e7d9a3e4813e0742f822ca9c8800cc3783a
Successfully built jellyfish cytoolz


In [ ]:
#import spacy
import textacy
SUBJ = ["nsubj","nsubjpass"] 
VERB = ["ROOT"] 
OBJ = ["dobj", "pobj", "dobj"] 
text = nlp(u'The cat sat on the mat. The cat jumped and picked up the biscuit. The cat ate biscuit and cookies.')
sub_toks = [tok for tok in text if (tok.dep_ in SUBJ) ]
obj_toks = [tok for tok in text if (tok.dep_ in OBJ) ]
vrb_toks = [tok for tok in text if (tok.dep_ in VERB) ]
text_ext = list(textacy.extract.subject_verb_object_triples(text))
print("Subjects:", sub_toks)
print("VERB :", vrb_toks)
print("OBJECT(s):", obj_toks)
print ("SVO:", text_ext)

Subjects: [cat, cat, cat]
VERB : [sat, jumped, ate]
OBJECT(s): [mat, biscuit, biscuit]
SVO: [(cat, ate, biscuit), (cat, ate, cookies)]
